In [1]:
import torch
import torch.nn as nn

torch.manual_seed(0)

# Fake logits và targets (batch=2, labels=3)
logits = torch.tensor([[0.0, 2.0, -1.0],
                       [3.0, -2.0, 0.5]], requires_grad=True)

targets = torch.tensor([[0., 1., 0.],
                        [1., 0., 1.]])

# -------------------------
# 1. Không có pos_weight
# -------------------------
criterion_no_pw = nn.BCEWithLogitsLoss(reduction='mean')

loss_no_pw = criterion_no_pw(logits, targets)
loss_no_pw.backward()

grad_no_pw = logits.grad.clone()  # lưu lại gradient
logits.grad.zero_()

# -------------------------
# 2. Có pos_weight
# -------------------------
pos_weight = torch.tensor([5., 5., 3.])  # mỗi label weight=5
criterion_pw = nn.BCEWithLogitsLoss(reduction='mean', pos_weight=pos_weight)

loss_pw = criterion_pw(logits, targets)
loss_pw.backward()

grad_pw = logits.grad.clone()

# -------------------------
# In kết quả
# -------------------------
print("=== Logits ===")
print(logits.detach())
print()

print("=== Targets ===")
print(targets)
print()

print("=== Gradient WITHOUT pos_weight ===")
print(grad_no_pw)
print()

print("=== Gradient WITH pos_weight ===")
print(grad_pw)
print()

print("=== Ratio (grad_pw / grad_no_pw) ===")
print(grad_pw / (grad_no_pw + 1e-9))  # tránh chia 0


=== Logits ===
tensor([[ 0.0000,  2.0000, -1.0000],
        [ 3.0000, -2.0000,  0.5000]])

=== Targets ===
tensor([[0., 1., 0.],
        [1., 0., 1.]])

=== Gradient WITHOUT pos_weight ===
tensor([[ 0.0833, -0.0199,  0.0448],
        [-0.0079,  0.0199, -0.0629]])

=== Gradient WITH pos_weight ===
tensor([[ 0.0833, -0.0993,  0.0448],
        [-0.0395,  0.0199, -0.1888]])

=== Ratio (grad_pw / grad_no_pw) ===
tensor([[1.0000, 5.0000, 1.0000],
        [5.0000, 1.0000, 3.0000]])


In [2]:
import torch
import torch.nn as nn

# --- 1. Class ASL đã thêm pos_weight (Code từ câu trước) ---
class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8,
                 disable_torch_grad_focal_loss=False, pos_weight=None):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        self.pos_weight = pos_weight
        if self.pos_weight is not None:
             if not isinstance(self.pos_weight, torch.Tensor):
                 self.pos_weight = torch.tensor(self.pos_weight)
             self.register_buffer('weight_buffer', self.pos_weight)
        else:
            self.weight_buffer = None

        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        self.targets = y
        self.anti_targets = 1 - y
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Tính Loss cơ bản
        pos_log = torch.log(self.xs_pos.clamp(min=self.eps))
        if self.weight_buffer is not None:
            pos_log = pos_log.mul(self.weight_buffer) # Nhân trọng số

        self.loss = self.targets * pos_log
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()

# --- 2. Hàm chạy Test so sánh ---
def run_test():
    print("=== TEST START: Comparing ASL with and without pos_weight ===\n")

    # GIẢ LẬP DỮ LIỆU
    # Batch size = 1, Num classes = 3
    # Logits = -2.0 -> Sigmoid(-2.0) ≈ 0.119 (Dự đoán thấp - Máy đang đoán sai)
    logits = torch.tensor([[-2.0, -2.0, -2.0]], requires_grad=True)

    # Target: Class 0 là Positive (Cần tìm), Class 1, 2 là Negative
    targets = torch.tensor([[1.0, 0.0, 0.0]])

    # ---------------------------------------------------------
    # TRƯỜNG HỢP 1: KHÔNG DÙNG POS_WEIGHT
    # ---------------------------------------------------------
    criterion_no_weight = AsymmetricLossOptimized(gamma_neg=4, gamma_pos=0, clip=0.05, pos_weight=None)

    # Forward
    loss_1 = criterion_no_weight(logits, targets)

    # Backward (Để xem Gradient)
    loss_1.backward()
    grad_1 = logits.grad.clone() # Lưu gradient lại
    logits.grad.zero_() # Reset gradient cho lần test sau

    print(f"[Case 1 - No Weight]")
    print(f"Loss Value: {loss_1.item():.4f}")
    print(f"Gradient tại Class 0 (Positive): {grad_1[0][0]:.4f}")
    print("-" * 40)

    # ---------------------------------------------------------
    # TRƯỜNG HỢP 2: CÓ POS_WEIGHT = 10 CHO CLASS 0
    # ---------------------------------------------------------
    # Ý nghĩa: Tôi muốn mô hình coi trọng Class 0 gấp 10 lần các class khác
    weights = torch.tensor([10.0, 1.0, 1.0])
    criterion_with_weight = AsymmetricLossOptimized(gamma_neg=4, gamma_pos=0, clip=0.05, pos_weight=weights)

    # Forward
    loss_2 = criterion_with_weight(logits, targets)

    # Backward
    loss_2.backward()
    grad_2 = logits.grad.clone()

    print(f"[Case 2 - With pos_weight=[10, 1, 1]]")
    print(f"Loss Value: {loss_2.item():.4f}")
    print(f"Gradient tại Class 0 (Positive): {grad_2[0][0]:.4f}")

    # ---------------------------------------------------------
    # SO SÁNH
    # ---------------------------------------------------------
    print("\n=== KẾT LUẬN ===")
    ratio = grad_2[0][0] / grad_1[0][0]
    print(f"Gradient tăng lên gấp: {ratio:.1f} lần")
    print("Điều này có nghĩa là mô hình sẽ học class 0 NHANH GẤP 10 LẦN so với bình thường.")

if __name__ == "__main__":
    run_test()

=== TEST START: Comparing ASL with and without pos_weight ===

[Case 1 - No Weight]
Loss Value: 2.1269
Gradient tại Class 0 (Positive): -0.8808
----------------------------------------
[Case 2 - With pos_weight=[10, 1, 1]]
Loss Value: 21.2693
Gradient tại Class 0 (Positive): -8.8080

=== KẾT LUẬN ===
Gradient tăng lên gấp: 10.0 lần
Điều này có nghĩa là mô hình sẽ học class 0 NHANH GẤP 10 LẦN so với bình thường.


In [3]:
import unittest
import torch
import torch.nn as nn
import math

# --- Import class loss của bạn vào đây ---
# Giả sử class AsymmetricLossOptimized đã được định nghĩa ở trên
# (Copy class AsymmetricLossOptimized vào đây hoặc import từ file khác)
class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8,
                 disable_torch_grad_focal_loss=False, pos_weight=None):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        self.pos_weight = pos_weight
        if self.pos_weight is not None:
             if not isinstance(self.pos_weight, torch.Tensor):
                 self.pos_weight = torch.tensor(self.pos_weight)
             self.register_buffer('weight_buffer', self.pos_weight)
        else:
            self.weight_buffer = None

        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        self.targets = y
        self.anti_targets = 1 - y
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        pos_log = torch.log(self.xs_pos.clamp(min=self.eps))
        if self.weight_buffer is not None:
            pos_log = pos_log.mul(self.weight_buffer)

        self.loss = self.targets * pos_log
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()


class TestAsymmetricLoss(unittest.TestCase):

    def setUp(self):
        # Setup chung: Logits và Targets giả lập
        self.logits = torch.tensor([[0.5, -2.0, 10.0], [-1.0, 5.0, -5.0]], requires_grad=True)
        self.targets = torch.tensor([[1.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
        # Batch size = 2, Num classes = 3

    def test_compare_with_bce_no_gamma_no_clip(self):
        """
        Kịch bản 1: Khi tắt hết gamma và clip, ASL phải giống hệt BCEWithLogitsLoss (reduction='sum').
        """
        print("\n--- Test 1: Compare with standard BCE ---")

        # ASL settings: gamma=0, clip=0 -> Trở về BCE thuần
        asl_loss = AsymmetricLossOptimized(gamma_neg=0, gamma_pos=0, clip=0)
        bce_loss = nn.BCEWithLogitsLoss(reduction='sum')

        loss_asl = asl_loss(self.logits, self.targets)
        loss_bce = bce_loss(self.logits, self.targets)

        print(f"ASL Loss: {loss_asl.item():.5f}")
        print(f"BCE Loss: {loss_bce.item():.5f}")

        # Kiểm tra độ sai lệch (dùng assert_close cho float)
        torch.testing.assert_close(loss_asl, loss_bce, rtol=1e-5, atol=1e-5)

    def test_pos_weight_logic(self):
        """
        Kịch bản 2: Kiểm tra pos_weight chỉ ảnh hưởng đến mẫu Dương, không ảnh hưởng mẫu Âm.
        """
        print("\n--- Test 2: Check pos_weight logic ---")

        # Chỉ xét 1 sample đơn giản: Class 0 (Pos), Class 1 (Neg)
        logits = torch.tensor([[0.0, 0.0]]) # Sigmoid(0) = 0.5
        targets = torch.tensor([[1.0, 0.0]])

        # Case A: Không weight
        criterion_base = AsymmetricLossOptimized(gamma_neg=0, gamma_pos=0, clip=0, pos_weight=None)
        loss_base = criterion_base(logits, targets)

        # Case B: Weight = 2 cho Class 0 (Pos)
        weights = torch.tensor([2.0, 10.0]) # 10.0 cho class 1 nhưng class 1 là target 0, nên k đc ảnh hưởng
        criterion_weighted = AsymmetricLossOptimized(gamma_neg=0, gamma_pos=0, clip=0, pos_weight=weights)
        loss_weighted = criterion_weighted(logits, targets)

        # Tính toán thủ công:
        # Loss Pos (Target=1, p=0.5) = -log(0.5) ≈ 0.693
        # Loss Neg (Target=0, p=0.5) = -log(1-0.5) ≈ 0.693
        # Base Total = 0.693 + 0.693 = 1.386

        # Weighted Total:
        # Pos được nhân 2 -> 0.693 * 2 = 1.386
        # Neg (dù weight=10 cũng ko đc nhân) -> 0.693
        # Total = 1.386 + 0.693 = 2.079

        print(f"Base Loss (Expected ~1.386): {loss_base.item():.4f}")
        print(f"Weighted Loss (Expected ~2.079): {loss_weighted.item():.4f}")

        self.assertTrue(loss_weighted > loss_base)
        # Kiểm tra tỷ lệ tăng có đúng logic không
        expected_increase = -math.log(0.5) # Lượng tăng thêm do weight=2 (thêm 1 lần log)
        diff = loss_weighted - loss_base
        self.assertAlmostEqual(diff.item(), expected_increase, places=4)

    def test_clipping_hard_threshold(self):
        """
        Kịch bản 3: Kiểm tra Clipping.
        Nếu mẫu Âm có xác suất p < clip, Loss phải bằng 0 tuyệt đối.
        """
        print("\n--- Test 3: Check Clipping (Hard Threshold) ---")

        # Logit = -10 -> Sigmoid(-10) ≈ 0.000045 (Rất nhỏ)
        logits = torch.tensor([[-10.0]], requires_grad=True)
        targets = torch.tensor([[0.0]]) # Negative sample

        # Clip = 0.05 (Lớn hơn xác suất dự đoán)
        criterion = AsymmetricLossOptimized(gamma_neg=0, gamma_pos=0, clip=0.05)
        loss = criterion(logits, targets)
        loss.backward()

        print(f"Logit: -10.0, Clip: 0.05")
        print(f"Loss Value (Expect 0.0): {loss.item()}")
        print(f"Gradient (Expect 0.0): {logits.grad.item()}")

        self.assertEqual(loss.item(), 0.0)
        self.assertEqual(logits.grad.item(), 0.0)

    def test_gamma_neg_attenuation(self):
        """
        Kịch bản 4: Kiểm tra Gamma Negative.
        Gamma càng cao thì Loss của mẫu dễ (easy negative) phải càng thấp.
        """
        print("\n--- Test 4: Check Gamma Negative Attenuation ---")

        logits = torch.tensor([[-2.0]]) # Sigmoid(-2) ≈ 0.12 (Easy Negative)
        targets = torch.tensor([[0.0]])

        # Gamma = 0
        crit_0 = AsymmetricLossOptimized(gamma_neg=0, clip=0)
        loss_0 = crit_0(logits, targets)

        # Gamma = 4
        crit_4 = AsymmetricLossOptimized(gamma_neg=4, clip=0)
        loss_4 = crit_4(logits, targets)

        print(f"Loss with Gamma=0: {loss_0.item():.5f}")
        print(f"Loss with Gamma=4: {loss_4.item():.5f}")

        # Loss 4 phải nhỏ hơn rất nhiều so với Loss 0
        self.assertTrue(loss_4 < loss_0)

        # Tính thủ công: Factor = (0.119)^4 ≈ 0.0002
        ratio = loss_4 / loss_0
        print(f"Attenuation Ratio (Loss4/Loss0): {ratio.item():.5f}")
        self.assertTrue(ratio < 0.01) # Giảm đi ít nhất 100 lần

    def test_numerical_stability(self):
        """
        Kịch bản 5: Kiểm tra tính ổn định số học (Numerical Stability).
        Với logits cực lớn hoặc cực nhỏ, Loss không được ra NaN.
        """
        print("\n--- Test 5: Check Numerical Stability ---")

        logits = torch.tensor([[100.0, -100.0]]) # Sigmoid(100) -> 1.0, Sigmoid(-100) -> 0.0
        targets = torch.tensor([[1.0, 0.0]])

        criterion = AsymmetricLossOptimized()
        loss = criterion(logits, targets)

        print(f"Loss with extreme logits: {loss.item()}")
        self.assertFalse(math.isnan(loss.item()))
        self.assertFalse(math.isinf(loss.item()))

    def test_batch_consistency(self):
        """
        Kịch bản 6: Kiểm tra tính đúng đắn khi chạy Batch.
        Tổng loss của batch phải bằng tổng loss của từng sample chạy riêng lẻ.
        """
        print("\n--- Test 6: Check Batch Consistency ---")

        l1 = torch.tensor([[0.5, -0.5]])
        t1 = torch.tensor([[1.0, 0.0]])

        l2 = torch.tensor([[1.5, -2.0]])
        t2 = torch.tensor([[0.0, 1.0]])

        criterion = AsymmetricLossOptimized()

        # Chạy riêng
        loss_1 = criterion(l1, t1)
        loss_2 = criterion(l2, t2)
        total_separate = loss_1 + loss_2

        # Chạy batch gộp
        l_batch = torch.cat([l1, l2], dim=0)
        t_batch = torch.cat([t1, t2], dim=0)
        loss_batch = criterion(l_batch, t_batch)

        print(f"Sum separate: {total_separate.item():.5f}")
        print(f"Batch loss: {loss_batch.item():.5f}")

        torch.testing.assert_close(loss_batch, total_separate, rtol=1e-5, atol=1e-5)

    def test_batch_pos_weight_logic(self):
        """
        Kịch bản 7: Kiểm tra pos_weight hoạt động đúng trên BATCH lớn.
        Đảm bảo broadcasting hoạt động chính xác (Trọng số class nào ăn vào class đó).
        """
        print("\n--- Test 7: Check Batch processing with pos_weight ---")

        # CẤU HÌNH DỮ LIỆU
        # Batch size = 2, Num Classes = 2
        # Logits = 0.0 -> Sigmoid(0.0) = 0.5
        # Việc dùng 0.5 giúp dễ tính nhẩm: -log(0.5) ≈ 0.6931
        logits = torch.zeros((2, 2))

        # Targets:
        # Sample 1: Class 0 là Dương, Class 1 là Âm
        # Sample 2: Class 0 là Âm, Class 1 là Dương
        targets = torch.tensor([
            [1.0, 0.0],
            [0.0, 1.0]
        ])

        # Weights:
        # Class 0: Trọng số 2.0
        # Class 1: Trọng số 3.0
        pos_weight = torch.tensor([2.0, 3.0])

        # KHỞI TẠO LOSS
        # Tắt gamma và clip để chỉ test logic của pos_weight + BCE
        criterion = AsymmetricLossOptimized(gamma_neg=0, gamma_pos=0, clip=0, pos_weight=pos_weight)

        # TÍNH TOÁN
        actual_loss = criterion(logits, targets)

        # TÍNH TAY (EXPECTED VALUE)
        base_loss = -math.log(0.5) # ≈ 0.693147

        # Sample 1:
        # - Class 0 (Pos, w=2): 2.0 * base_loss
        # - Class 1 (Neg, ko w): 1.0 * base_loss
        loss_s1 = (2.0 * base_loss) + base_loss

        # Sample 2:
        # - Class 0 (Neg, ko w): 1.0 * base_loss
        # - Class 1 (Pos, w=3): 3.0 * base_loss
        loss_s2 = base_loss + (3.0 * base_loss)

        expected_total_loss = loss_s1 + loss_s2

        print(f"Sample 1 Loss (Expected ~ {3 * 0.693:.2f}): {loss_s1:.4f}")
        print(f"Sample 2 Loss (Expected ~ {4 * 0.693:.2f}): {loss_s2:.4f}")
        print(f"Total Batch Loss: {actual_loss.item():.4f}")

        # SO SÁNH
        # Kiểm tra xem code có ra đúng con số (2+1 + 1+3) = 7 lần base_loss không
        self.assertAlmostEqual(actual_loss.item(), expected_total_loss, places=4)

if __name__ == '__main__':
    # argv=['first-arg-is-ignored']: Bỏ qua các tham số hệ thống của Colab
    # exit=False: Ngăn không cho Colab bị tắt (crash) sau khi chạy xong test
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....


--- Test 6: Check Batch Consistency ---
Sum separate: 2.56346
Batch loss: 2.56346

--- Test 7: Check Batch processing with pos_weight ---
Sample 1 Loss (Expected ~ 2.08): 2.0794
Sample 2 Loss (Expected ~ 2.77): 2.7726
Total Batch Loss: 4.8520

--- Test 3: Check Clipping (Hard Threshold) ---
Logit: -10.0, Clip: 0.05
Loss Value (Expect 0.0): -0.0
Gradient (Expect 0.0): -0.0

--- Test 1: Compare with standard BCE ---
ASL Loss: 0.92774
BCE Loss: 0.92774

--- Test 4: Check Gamma Negative Attenuation ---
Loss with Gamma=0: 0.12693
Loss with Gamma=4: 0.00003
Attenuation Ratio (Loss4/Loss0): 0.00020


..
----------------------------------------------------------------------
Ran 7 tests in 0.026s

OK



--- Test 5: Check Numerical Stability ---
Loss with extreme logits: -0.0

--- Test 2: Check pos_weight logic ---
Base Loss (Expected ~1.386): 1.3863
Weighted Loss (Expected ~2.079): 2.0794
